## speak onetime if person is apper

In [1]:
import cv2
import os
import time
import pyttsx3

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Define paths to the model files
prototxt_path = r'gender_pre_trained_models\gender_deploy.prototxt'
caffemodel_path = r'gender_pre_trained_models\gender_net.caffemodel'

# Check if files exist
if not os.path.isfile(prototxt_path):
    raise FileNotFoundError(f"Cannot find the prototxt file at {prototxt_path}")
if not os.path.isfile(caffemodel_path):
    raise FileNotFoundError(f"Cannot find the caffemodel file at {caffemodel_path}")

# Load the pre-trained Caffe models for gender classification
gender_net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Define the mean values for the gender model
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

# Gender list
gender_list = ['Male', 'Female']

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the webcam
cap = cv2.VideoCapture(0)

# Initialize variables to keep track of detection and greeting logic
last_detect_time = 0
last_greet_time = 0
greeting_interval = 2  # Interval to check if person is not available
greeted = False

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        last_detect_time = time.time()
        if not greeted:
            for (x, y, w, h) in faces:
                # Extract the face region
                face_img = frame[y:y+h, x:x+w].copy()

                # Prepare the face image for classification
                blob = cv2.dnn.blobFromImage(face_img, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
                gender_net.setInput(blob)

                # Perform gender classification
                gender_preds = gender_net.forward()
                gender = gender_list[gender_preds[0].argmax()]

                # Draw a rectangle around the face and display the gender
                label = f'{gender}'
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                # Say the greeting based on detected gender
                if gender == 'Male':
                    engine.say("Hello Sir")
                elif gender == 'Female':
                    engine.say("Hello Miss")
                engine.runAndWait()
                greeted = True
    else:
        # Check if the person has been missing for more than the greeting interval
        if time.time() - last_detect_time >= greeting_interval:
            greeted = False

    # Display the resulting frame
    cv2.imshow('Gender Classification', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


## good morning

In [2]:
from gtts import gTTS
import datetime
import pytz
import os
import time

def get_greeting():
    # Define the timezone for Sri Lanka
    sri_lanka_tz = pytz.timezone('Asia/Colombo')
    
    # Get the current time in Sri Lanka
    now = datetime.datetime.now(sri_lanka_tz)
    
    # Determine the greeting based on the current time
    hour = now.hour
    if 5 <= hour < 12:
        return "Good morning"
    elif 12 <= hour < 18:
        return "Good afternoon"
    else:
        return "Good evening"

def speak(text):
    # Convert text to speech
    tts = gTTS(text=text, lang='en')
    audio_file = 'greeting.mp3'
    tts.save(audio_file)
    
    # Play the audio file
    os.system(f"start {audio_file}")  # Use "start" for Windows, "open" for macOS, "xdg-open" for Linux

def main():
    # Define the interval for repeating greetings
    greeting_interval = 60 * 60  # 1 hour
    last_greeting_time = 0
    
    while True:
        current_time = time.time()
        
        # Check if it's time to speak a new greeting
        if current_time - last_greeting_time >= greeting_interval:
            greeting = get_greeting()
            speak(greeting)
            last_greeting_time = current_time
        
        # Sleep for a short time to avoid high CPU usage
        time.sleep(60)  # Check every minute

if __name__ == "__main__":
    main()


## combine code

In [9]:
import cv2
import os
import time
import pyttsx3
import datetime
import pytz

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Define paths to the model files
prototxt_path = r'gender_pre_trained_models\gender_deploy.prototxt'
caffemodel_path = r'gender_pre_trained_models\gender_net.caffemodel'

# Check if files exist
if not os.path.isfile(prototxt_path):
    raise FileNotFoundError(f"Cannot find the prototxt file at {prototxt_path}")
if not os.path.isfile(caffemodel_path):
    raise FileNotFoundError(f"Cannot find the caffemodel file at {caffemodel_path}")

# Load the pre-trained Caffe models for gender classification
gender_net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Define the mean values for the gender model
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

# Gender list
gender_list = ['Male', 'Female']

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the webcam
cap = cv2.VideoCapture(0)

# Initialize variables to keep track of detection and greeting logic
last_detect_time = 0
last_greet_time = 0
greeting_interval = 2  # Interval to check if person is not available
greeted = False

def get_time_based_greeting():
    # Define the timezone for Sri Lanka
    sri_lanka_tz = pytz.timezone('Asia/Colombo')
    
    # Get the current time in Sri Lanka
    now = datetime.datetime.now(sri_lanka_tz)
    
    # Determine the greeting based on the current time
    hour = now.hour
    if 5 <= hour < 12:
        return "Good morning"
    elif 12 <= hour < 18:
        return "Good afternoon"
    else:
        return "Good evening"

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        last_detect_time = time.time()
        if not greeted:
            for (x, y, w, h) in faces:
                # Extract the face region
                face_img = frame[y:y+h, x:x+w].copy()

                # Prepare the face image for classification
                blob = cv2.dnn.blobFromImage(face_img, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
                gender_net.setInput(blob)

                # Perform gender classification
                gender_preds = gender_net.forward()
                gender = gender_list[gender_preds[0].argmax()]

                # Draw a rectangle around the face and display the gender
                label = f'{gender}'
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                # Get the time-based greeting
                time_based_greeting = get_time_based_greeting()
                
                # Say the greeting based on detected gender
                if gender == 'Male':
                    engine.say(f"{time_based_greeting} Sir")
                elif gender == 'Female':
                    engine.say(f"{time_based_greeting} Miss")
                engine.runAndWait()
                greeted = True
    else:
        # Check if the person has been missing for more than the greeting interval
        if time.time() - last_detect_time >= greeting_interval:
            greeted = False

    # Display the resulting frame
    cv2.imshow('Gender Classification', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


## updated one | say greeting with sir/miss

In [4]:
import cv2
import os
import time
import pyttsx3
import datetime
import pytz

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Define paths to the model files
prototxt_path = r'gender_pre_trained_models\gender_deploy.prototxt'
caffemodel_path = r'gender_pre_trained_models\gender_net.caffemodel'

# Check if files exist
if not os.path.isfile(prototxt_path):
    raise FileNotFoundError(f"Cannot find the prototxt file at {prototxt_path}")
if not os.path.isfile(caffemodel_path):
    raise FileNotFoundError(f"Cannot find the caffemodel file at {caffemodel_path}")

# Load the pre-trained Caffe models for gender classification
gender_net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Define the mean values for the gender model
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

# Gender list
gender_list = ['Male', 'Female']

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the webcam
cap = cv2.VideoCapture(0)

# Initialize variables to keep track of detection and greeting logic
last_detect_time = 0
greeting_interval = 2  # Interval to check if person is not available
greeted = False

def get_time_based_greeting():
    # Define the timezone for Sri Lanka
    sri_lanka_tz = pytz.timezone('Asia/Colombo')
    
    # Get the current time in Sri Lanka
    now = datetime.datetime.now(sri_lanka_tz)
    
    # Determine the greeting based on the current time
    hour = now.hour
    if 5 <= hour < 12:
        return "Good morning"
    elif 12 <= hour < 18:
        return "Good afternoon"
    else:
        return "Good evening"

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        last_detect_time = time.time()

        if not greeted:
            for (x, y, w, h) in faces:
                # Extract the face region
                face_img = frame[y:y+h, x:x+w].copy()

                # Prepare the face image for classification
                blob = cv2.dnn.blobFromImage(face_img, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
                gender_net.setInput(blob)

                # Perform gender classification
                gender_preds = gender_net.forward()
                gender = gender_list[gender_preds[0].argmax()]

                # Draw a rectangle around the face and display the gender
                label = f'{gender}'
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                # Get the time-based greeting
                time_based_greeting = get_time_based_greeting()
                
                # Say the greeting based on detected gender
                if gender == 'Male':
                    engine.say(f"{time_based_greeting} Sir")
                elif gender == 'Female':
                    engine.say(f"{time_based_greeting} Miss")
                engine.runAndWait()
                
                # Set greeted to True to prevent repeated greetings
                greeted = True
                break  # Exit the loop after greeting
    else:
        # Check if the person has been missing for more than the greeting interval
        if time.time() - last_detect_time >= greeting_interval:
            greeted = False

    # Display the resulting frame
    cv2.imshow('Gender Classification', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


## gsheet reading

In [8]:
import os
import pyttsx3
from google.oauth2 import service_account
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Path to your OAuth 2.0 credentials file
CREDENTIALS_FILE = 'credentials.json'
# Scopes required for Google Sheets API
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

def get_google_sheets_service():
    """Builds a Google Sheets API service object using OAuth 2.0."""
    flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
    creds = flow.run_local_server(port=0)  # Opens a browser window for user authorization
    service = build('sheets', 'v4', credentials=creds)
    return service

def read_sheet_data(service, spreadsheet_id, range_name):
    """Reads data from a Google Sheet."""
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    values = result.get('values', [])
    if not values:
        print('No data found.')
    return values

def write_sheet_data(service, spreadsheet_id, range_name, values):
    """Writes data to a Google Sheet."""
    sheet = service.spreadsheets()
    body = {
        'values': values
    }
    result = sheet.values().update(spreadsheetId=spreadsheet_id, range=range_name, valueInputOption='RAW', body=body).execute()
    return result

def speak(text):
    """Speaks the provided text."""
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

def main():
    service = get_google_sheets_service()
    
    # Replace with your Google Sheets ID and range
    spreadsheet_id = '1lNZDwXKNlNkZiLWKksob55RE70e3IbSAhkR0W_pjGV4'
    range_name = 'Sheet1!A1:C'  # Adjust the range as needed
    
    # Read data from the Google Sheet
    data = read_sheet_data(service, spreadsheet_id, range_name)
    
    # Speak the details
    for row in data:
        if len(row) >= 3:  # Ensure there are at least 3 columns
            product = row[0]
            brand_type = row[1]
            price = row[2]
            details = f"Product: {product}, Brand and Type: {brand_type}, Price: {price}"
            print(details)  # Optional: Print to console
            speak(details)  # Read out the details

    # Example data to write
    new_values = [
        ["New Product", "New Brand", "New Price"],
        ["Another Product", "Another Brand", "Another Price"]
    ]
    
    # Write new data to the Google Sheet
    write_range = 'Sheet1!A4:C'  # Adjust the range where you want to write the new data
    result = write_sheet_data(service, spreadsheet_id, write_range, new_values)
    print(f"Cells updated: {result.get('updatedCells')}")

if __name__ == '__main__':
    main()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=965765739038-bsu15r84t2433o5ncsj3fnuktd39q13u.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64558%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=l1DnhPlkqEaPLde5jhafG94uOWsnep&access_type=offline
Product: Product, Brand and Type: BrandType, Price: Price
Product: TV, Brand and Type: samsung 32'', Price: Rs.38000
Product: gas cooker , Brand and Type: usha , Price: Rs.15000
Product: New Product, Brand and Type: New Brand, Price: New Price
Product: Another Product, Brand and Type: Another Brand, Price: Another Price
Cells updated: 6


## without accesing g account

In [5]:
import os
import pickle
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Replace with your Google Sheets ID and range
SPREADSHEET_ID = '1lNZDwXKNlNkZiLWKksob55RE70e3IbSAhkR0W_pjGV4'  # Extracted from the URL
READ_RANGE = 'Sheet1!A1:C'  # Adjust the range as needed

# Path to your credentials.json file
CREDENTIALS_FILE = 'credentials.json'
TOKEN_FILE = 'token.pickle'

def get_google_sheets_service():
    """Builds a Google Sheets API service object."""
    creds = None

    # Load credentials from file if they exist
    if os.path.exists(TOKEN_FILE):
        with open(TOKEN_FILE, 'rb') as token:
            creds = pickle.load(token)

    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CREDENTIALS_FILE, 
                scopes=['https://www.googleapis.com/auth/spreadsheets.readonly']
            )
            creds = flow.run_local_server(port=0)

        # Save the credentials for the next run
        with open(TOKEN_FILE, 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)
    return service

def read_sheet_data(service, spreadsheet_id, range_name):
    """Reads data from a Google Sheet."""
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    values = result.get('values', [])
    if not values:
        print('No data found.')
    return values

def main():
    service = get_google_sheets_service()

    # Read data from the Google Sheet
    data = read_sheet_data(service, SPREADSHEET_ID, READ_RANGE)

    # Print the data
    for row in data:
        print(row)

if __name__ == '__main__':
    main()


['Product', 'BrandType', 'Price']
['TV', "samsung 32''", 'Rs.38000']
['gas cooker ', 'usha ', 'Rs.15000']
['New Product', 'New Brand', 'New Price']
['Another Product', 'Another Brand', 'Another Price']


## normal accesing public sheet

In [8]:
import requests
import pandas as pd
from io import StringIO

# Replace with your Google Sheets URL
SPREADSHEET_ID = '1lNZDwXKNlNkZiLWKksob55RE70e3IbSAhkR0W_pjGV4'
SHEET_NAME = 'Sheet1'  # Adjust if necessary

# Construct the URL for the Google Sheets API
url = f'https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'

def fetch_data_from_sheet(url):
    """Fetches CSV data from the Google Sheet URL."""
    response = requests.get(url)
    if response.status_code == 200:
        return response.content.decode('utf-8')
    else:
        print('Failed to fetch data.')
        return None

def main():
    csv_data = fetch_data_from_sheet(url)
    if csv_data:
        # Convert CSV data to DataFrame using StringIO
        df = pd.read_csv(StringIO(csv_data))
        print(df)

if __name__ == '__main__':
    main()


           Product      BrandType          Price
0               TV   samsung 32''       Rs.38000
1      gas cooker           usha        Rs.15000
2      New Product      New Brand      New Price
3  Another Product  Another Brand  Another Price


## getting user inputs and answer the quension

In [12]:
import requests
import pandas as pd
from io import StringIO
import pyttsx3
import speech_recognition as sr

# Google Sheets configuration
SPREADSHEET_ID = '1lNZDwXKNlNkZiLWKksob55RE70e3IbSAhkR0W_pjGV4'
SHEET_NAME = 'Sheet1'  # Adjust if necessary
URL = f'https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'

def fetch_data_from_sheet(url):
    """Fetches CSV data from the Google Sheet URL."""
    response = requests.get(url)
    if response.status_code == 200:
        return response.content.decode('utf-8')
    else:
        print('Failed to fetch data.')
        return None

def speak(text):
    """Speaks the provided text."""
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

def get_user_input():
    """Gets user input via speech recognition."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio)
            print(f"You said: {text}")
            return text
        except sr.UnknownValueError:
            print("Sorry, I did not understand that.")
            return ""
        except sr.RequestError:
            print("Sorry, there was an issue with the speech recognition service.")
            return ""

def main():
    # Greet the user
    speak("How can I help you?")
    
    # Get user input
    user_input = get_user_input()
    
    if not user_input:
        return

    # Extract keywords from user input
    keywords = ['tv', 'gas cooker']  # Add more keywords as needed
    matched_keywords = [keyword for keyword in keywords if keyword in user_input.lower()]

    if not matched_keywords:
        speak("Sorry, I couldn't find any matching items.")
        return

    # Fetch data from Google Sheets
    csv_data = fetch_data_from_sheet(URL)
    if not csv_data:
        speak("Failed to fetch data from the Google Sheet.")
        return
    
    # Convert CSV data to DataFrame
    df = pd.read_csv(StringIO(csv_data))
    
    # Check for matches in DataFrame
    for keyword in matched_keywords:
        matched_rows = df[df['Product'].str.contains(keyword, case=False, na=False)]
        if not matched_rows.empty:
            for _, row in matched_rows.iterrows():
                product = row['Product']
                brand_type = row['BrandType']
                price = row['Price']
                details = f"We have {product}, {brand_type} with the price {price}"
                speak(details)
        else:
            speak(f"Sorry, we don't have information on {keyword}.")

if __name__ == '__main__':
    main()


Listening...
You said: I won't buy gas cooker


## update google sheet accesing

In [14]:
import requests
import pandas as pd
from io import StringIO
import pyttsx3
import speech_recognition as sr

# Google Sheets configuration
SPREADSHEET_ID = '1lNZDwXKNlNkZiLWKksob55RE70e3IbSAhkR0W_pjGV4'
SHEET_NAME = 'Sheet1'  # Adjust if necessary
URL = f'https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'

def fetch_data_from_sheet(url):
    """Fetches CSV data from the Google Sheet URL."""
    response = requests.get(url)
    if response.status_code == 200:
        return response.content.decode('utf-8')
    else:
        print('Failed to fetch data.')
        return None

def speak(text):
    """Speaks the provided text."""
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

def get_user_input():
    """Gets user input via speech recognition."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio)
            print(f"You said: {text}")
            return text
        except sr.UnknownValueError:
            print("Sorry, I did not understand that.")
            return ""
        except sr.RequestError:
            print("Sorry, there was an issue with the speech recognition service.")
            return ""

def preprocess_data(df):
    """Preprocess the DataFrame to handle empty rows and group product details."""
    df = df.fillna('')
    df['Product'] = df['Product'].replace('', pd.NA).ffill()
    return df.dropna(subset=['Product'])

def main():
    # Greet the user
    speak("How can I help you?")
    
    # Get user input
    user_input = get_user_input()
    
    if not user_input:
        return

    # Extract keywords from user input
    keywords = ['tv', 'gas cooker', 'fridge']  # Add more keywords as needed
    matched_keywords = [keyword for keyword in keywords if keyword in user_input.lower()]

    if not matched_keywords:
        speak("Sorry, I couldn't find any matching items.")
        return

    # Fetch data from Google Sheets
    csv_data = fetch_data_from_sheet(URL)
    if not csv_data:
        speak("Failed to fetch data from the Google Sheet.")
        return
    
    # Convert CSV data to DataFrame
    df = pd.read_csv(StringIO(csv_data))
    
    # Preprocess the data
    df = preprocess_data(df)
    
    # Check for matches in DataFrame
    for keyword in matched_keywords:
        matched_rows = df[df['Product'].str.contains(keyword, case=False, na=False)]
        if not matched_rows.empty:
            for _, row in matched_rows.iterrows():
                product = row['Product']
                brand_type = row['BrandType']
                price = row['Price']
                details = f"We have {product}, {brand_type} with the price {price}"
                speak(details)
        else:
            speak(f"Sorry, we don't have information on {keyword}.")

if __name__ == '__main__':
    main()


Listening...
You said: goodbye fridge
